In [1]:
import sys, os

sys.path.append("/home/funny/Documents/Projects/Augustin-Parser/augustin-plugin/")
from dotenv import load_dotenv
import fitz

In [2]:
# Load environment variables
load_dotenv()
global_save_path = os.environ.get("SAVE_PATH")
path_to_file = (
    global_save_path + "570_augustin_fertig_kontrolle/570_augustin_fertig_kontrolle.pdf"
)
src = fitz.open(path_to_file)

In [3]:
def extract_headlines(
    page, starting_characters=None, headlines=None, searching_for_end=False
):
    """Extract headlines from a PDF page."""
    ending_symbols = 0
    text_instances = page.get_text("dict", sort=True)["blocks"]

    if not text_instances:
        return headlines, starting_characters, ending_symbols

    for text in text_instances:
        try:
            for line in text["lines"]:
                for span in line["spans"]:
                    font_size = span["size"]
                    colour_code = f"#{span['color']:06x}"

                    # Check if colour code is not black or dark brown since starting characters have version colour
                    if colour_code != "#2e2013" and colour_code != "#000000":
                        # ------ Starting characters check ------
                        # Check if text is not a number, has less than 3 characters and is not ending character
                        if (
                            not span["text"].isdigit()
                            and len(span["text"]) < 3
                            and span["text"] != "■"
                        ):
                            print(f"Extracting starting character: {span['text']}")
                            if starting_characters is None:
                                starting_characters = []
                            if not searching_for_end:
                                starting_characters.append(span["text"].strip())
                        # ------ Ending symbol check ------
                        elif "■" in span["text"]:
                            print(f"Extracting ending symbol: {span['text']}")
                            ending_symbols += 1

                    # ------ Header check ------
                    # Check if font is bold and font size is larger 12
                    elif font_size > 12 or span["font"] == "AmasisMTStd-Bold":
                        print(f"Extracting headline: {span['text'].strip()}")
                        if headlines is None:
                            headlines = []
                        if not searching_for_end:
                            headlines.append(span["text"].strip())
        except KeyError:
            pass

    return headlines, starting_characters, ending_symbols

In [14]:
def clean_text(raw_text, starting_characters):
    """Clean the text from unwanted newlines and hyphens."""
    found_starting_character = False

    article = ""

    for line in raw_text.split("\n"):
        if found_starting_character:
            # Clean text from unwanted hyphens and add newlines, if line is not empty
            if line:
                try:
                    if line[-1] == " ":
                        if line[-2] in [".", "!", "?", ":"]:
                            article += line[:-1] + "\n"
                            continue
                        article += line
                        continue
                    # If in the end of line is "." or "!" or "?" or ":" keep newline
                    if line[-1] in [".", "!", "?", ":"]:
                        article += line + "\n"
                        continue
                    # If in the end of line is "-" delete hyphen and add line to article
                    elif line[-1] == "-":
                        article += line[:-1]
                        continue
                except IndexError:
                    pass

            # If no checks are met, add line to article
            article += line
        # Start extracting article text with first starting character
        if starting_characters[0] == line:
            article += line
            found_starting_character = True

    # Format the string
    article = list(article)
    article_edit = article
    for index, letter in enumerate(article):
        if letter == "■":
            print("found ending char")
            del article_edit[index:]
        if " " in letter and " " in article[index - 1]:
            del article_edit[index]

    return "".join(article_edit)

In [15]:
for index, page in enumerate(src):
    if index == 8:
        headlines, starting_characters, ending_symbols = extract_headlines(page)
        article = clean_text(page.get_text("text"), starting_characters)
        print(f"Article: {article}")

Extracting headline: 9
Extracting headline: Burg-Friede
Extracting starting character: D
Extracting ending symbol: ■
Extracting headline: Neutrale Zonen
Extracting headline: «Friedensarbeit ist
Extracting headline: nichts, was sofort
Extracting headline: Früchte trägt»
Die öffentliche Anreise zur Friedensburg Schlaining hat etwas von einem längeren Friedensmarsch. Sie liegt näher bei  Szombathely als bei Wiener Neustadt, östlich von Oberwart, wohin leider keine Züge fahren. Mit dem Bus ist man von Wien aus gute zwei Stunden unterwegs – um dann festzustellen, dass das Friedensmuseum im Jahr 2019 der Burgenländischen Landesausstellung gewichen ist.
 Dafür aber ist die Burg der Hauptsitz des Austrian Centre for Peace (ACP), vormals  Austrian Study Centre for Peace and Conflict Resolution (ASPR, Österreichisches Studienzentrum für Frieden und Konfliktlösungen). «Wir sind ein Friedenszentrum», sagt Lukas Wank, scheidender stellvertretender Direktor des ACP, der unter anderem das Kooperation